[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2024/blob/main/99--exercises/23--cars.ipynb)

## Jobs 

In [ ]:
!mkdir -p var/

In [ ]:
!wget https://data.edu.jkanclerz.com/data-science/uek/datasets/cars.zip -O var/cars.zip

In [ ]:
!unzip var/cars.zip -d var/

In [ ]:
pip install pandas

In [ ]:
import glob
import json
import pandas as pd 

In [ ]:
cars = glob.glob('./var/cars/**/*.json', recursive=True)

In [ ]:
cars_as_json = (json.load(open(path)) for path in cars)

In [ ]:
df = pd.DataFrame(cars_as_json)
data = df

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df = data

In [ ]:
df = df.rename(columns={
    "Marka pojazdu": "marka",
    "Model pojazdu": "model",
    "Rok produkcji": "rok_produkcji",
    "Zarejestrowany w Polsce": "rejstracja_polska",
    "Przebieg": "przebieg",
    "Rodzaj paliwa": "paliwo",
    "Kraj pochodzenia": "kraj_pochodzenia",
    "Pierwszy właściciel": "prierwszy_wlasciciel"
})

In [ ]:
df.columns

In [ ]:
df.head()

## Understand the data by observing a few rows

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.tail()

### dtypes, values, shape

In [ ]:
df.dtypes

In [ ]:
df.describe().T

In [ ]:
df = df[['id', 'marka', 'model', 'price', 'rok_produkcji', 'przebieg', "kraj_pochodzenia", "paliwo", "prierwszy_wlasciciel"]]

In [ ]:
df.dtypes

## Clearing transformations

In [ ]:
df = df.replace('', None)

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()

In [ ]:
def clear_millage(millage_as_str):
    return int(str(millage_as_str)
               .replace('km', '')
               .replace(' ', '')
           )

assert clear_millage('172 000 km') == 172000

In [ ]:
df['przebieg'] = df['przebieg'].apply(clear_millage)

In [ ]:
df['price'] = df['price'].apply(float)
df['rok_produkcji'] = df['rok_produkcji'].apply(int)

In [ ]:
df['year_as_dt'] = pd.to_datetime(df['rok_produkcji'], format='%Y')

In [ ]:
df.describe().T

In [ ]:
df['przebieg'].quantile(q=0.95), df['przebieg'].quantile(q=0.99)

In [ ]:
df['price'].quantile(q=0.95), df['price'].quantile(q=0.99)

In [ ]:
df = df[df['przebieg'] < df['przebieg'].quantile(q=0.99)]

In [ ]:
df.dtypes

## Save intermediate

In [ ]:
df.to_csv("var/cars.csv")

In [ ]:
loaded_csv = pd.read_csv("var/cars.csv")

In [ ]:
loaded_csv.dtypes

In [ ]:
pip install fastparquet pyarrow

In [ ]:
df.to_parquet('cars.parquet.gzip', compression='gzip')

In [ ]:
loaded_parquet = pd.read_parquet('cars.parquet.gzip')

In [ ]:
loaded_parquet.dtypes

In [ ]:
cars = loaded_parquet

In [ ]:
cars.describe().T

In [ ]:
cars[['marka', 'model']].describe()

## Overview single feature - Univariate Analysis

In [ ]:
pip install matplotlib seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
def histogram_boxplot(data, feature, figsize = (12, 7), kde = False, bins = None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (12,7))
    kde: whether to show density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows = 2,     
        sharex = True,
        gridspec_kw = {"height_ratios": (0.25, 0.75)},
        figsize = figsize,
    )  
    sns.boxplot(
        data = data, x = feature, ax = ax_box2, showmeans = True, color = "blue"
    ) 
    sns.histplot(
        data = data, x = feature, kde = kde, ax = ax_hist2, bins = bins, palette = "winter"
    ) if bins else sns.histplot(
        data = data, x = feature, kde = kde, ax = ax_hist2
    )                   # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color = "green", linestyle = "--"
    )                   # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color = "red", linestyle = "-"
    )   

In [ ]:
histogram_boxplot(cars, 'przebieg')

In [ ]:
histogram_boxplot(cars, 'price')

In [ ]:
sns.displot(np.log(cars["price"]), kde=False, height=7, aspect=12/7)
plt.show()

### transform to log? 

In [ ]:
cars['price_log'] = np.log(cars['price'])

In [ ]:
cars

## Univariate analysis - Categorical Data

In [ ]:
category_counts = df['marka'].value_counts()
top_brands = cars[cars['marka'].isin(category_counts.head(15).index)]

In [ ]:
plt.figure(figsize=(12, 7)) 

sns.countplot(x='marka', data=top_brands, hue='marka')

plt.xticks(rotation=90)
plt.title('Distribution of Brand Variable')
plt.xlabel('Brand')
plt.ylabel('Count')

In [ ]:
def perc_on_bar(data, z):
    total = len(data[z])
    plt.figure(figsize=(12, 5))
    palette = sns.color_palette("Set2", n_colors=data[z].nunique())
           
    ax = sns.countplot(data=data, x=z, hue=z, order=data[z].value_counts().index)
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2 - 0.05
        y = p.get_y() + p.get_height() - 1

        ax.annotate(percentage, (x, y), size=12, ha='center')  # Annotate the percentage with center alignment
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
perc_on_bar(top_brands, 'marka')

In [ ]:
plt.figure(figsize=(12, 7)) 

sns.countplot(x='paliwo', data=cars, hue="paliwo")

plt.xticks(rotation=90)

## Bivariate Analysis

In [ ]:
cars

### Scatter plot

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 7))

cars.plot(x = 'price_log', y = 'year_as_dt', style = '.', ax=axes[0])
axes[0].set_title('Price vs Year')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Year')

cars.plot(x = 'price', y = 'year_as_dt', style = '.', ax=axes[1])
axes[1].set_title('Log(Price) vs Year')
axes[1].set_xlabel('Log(Price)')
axes[1].set_ylabel('Year')




In [ ]:
cars.plot(x = 'przebieg', y = 'year_as_dt', style = '.')

In [ ]:
cars.plot(x = 'przebieg', y = 'price', style = '.')

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
plt.figure(figsize=(12, 8))
scatter_matrix(cars, figsize=(12, 8), diagonal='hist', alpha=0.7)

## Heat map

In [ ]:
plt.figure(figsize = (12, 7))

sns.heatmap(cars.drop(['model', 'marka', 'id'], axis = 1).corr(numeric_only = True), annot = True, vmin = -1, vmax = 1)

plt.show()

## Box plot

In [ ]:
def boxplot(z):
    plt.figure(figsize = (12, 5))
    sns.boxplot(x = z, y = cars['price'])
    plt.yticks(range(0, int(cars['price'].max()) + 1000000, 1000000))
    plt.show()
    
    plt.figure(figsize = (12, 5))
    plt.title('No Outliers')
    sns.boxplot(x = z, y = cars['price'], showfliers = False)
    plt.show()

In [ ]:
boxplot(cars['paliwo'])

### Building Various Models

In [ ]:
cars.head(1)

In [ ]:
data = cars[['marka', 'rok_produkcji', 'przebieg', 'paliwo']]

In [ ]:
data.describe().T

In [ ]:
data[['paliwo']].describe()

In [ ]:
pd.get_dummies(data, drop_first=True).head(1)

In [ ]:
pd.get_dummies(data).head(1)

In [ ]:
X = pd.get_dummies(data, drop_first=True)

In [ ]:
y = cars[['price_log', 'price']]

In [ ]:
pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Step-3 Splitting data into training and test set:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

(X_train.shape, X_test.shape), (y_train.shape, y_test.shape)

In [ ]:
from sklearn import metrics

def get_model_score(model, flag = True):

    pred_train = model.predict(X_train)
    pred_train_ = np.exp(pred_train)
    pred_test = model.predict(X_test)
    pred_test_ = np.exp(pred_test)
    
    score_list = []
    train_r2 = metrics.r2_score(y_train['price'], pred_train_)
    test_r2 = metrics.r2_score(y_test['price'], pred_test_)
    train_rmse = metrics.mean_squared_error(y_train['price'], pred_train_)
    test_rmse = metrics.mean_squared_error(y_test['price'], pred_test_)
    score_list.extend((train_r2, test_r2, train_rmse, test_rmse))

    print("R-sqaure on training set : ", metrics.r2_score(y_train['price'], pred_train_))
    print("R-square on test set : ", metrics.r2_score(y_test['price'], pred_test_))
    print("RMSE on training set : ", np.sqrt(metrics.mean_squared_error(y_train['price'], pred_train_)))
    print("RMSE on test set : ", np.sqrt(metrics.mean_squared_error(y_test['price'], pred_test_)))

    # Returning the list with train and test scores
    return score_list

### LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train['price_log'])

In [ ]:
LR_score = get_model_score(lr)

In [ ]:
X.sample(3, random_state=1)

In [ ]:
to_be_predicted = X.sample(1, random_state=1)

In [ ]:
to_be_predicted

In [ ]:
[np.exp(value) for value in lr.predict(to_be_predicted)]

In [ ]:
y.iloc[8090]

### Decision Tree
https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html

### Random Forest
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html